<a href="https://colab.research.google.com/github/sfoucher/Masterclass-Biodiversity/blob/code-reorg/jupyter_notebooks/colab_setup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installing mmdetection in colab can be tricky.

First make sure you have a GPU:

In [1]:
!nvidia-smi

Thu Dec  2 13:51:12 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    29W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# Vérification de la version de NVIDIA CUDA Compiler
print('-- Checking nvcc version...')
!nvcc -V

# Vérification de la version de GPU Computer Collection
print('\n-- Checking GCC version...')
!gcc --version

-- Checking nvcc version...
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0

-- Checking GCC version...
gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



From the information above set your CUDA version:    

`CUDA= 'cu111'`

**Step 1**: Check the following site for available wheels for mmcv:

https://openmmlab.oss-accelerate.aliyuncs.com/mmcv/dist/index.html

Find the right combination of cuda, torch and torchvision:

In [3]:
CUDA= 'cu111'
t_version= '1.8.0'
tv_version= '0.9.0'

Setting a few env variables (seems to help):

In [4]:
%env FORCE_CUDA='1'
%env TORCH_CUDA_ARCH_LIST="6.0 6.1 7.0+PTX"
%env TORCH_NVCC_FLAGS="-Xfatbin -compress-all"
%env CMAKE_PREFIX_PATH="$(dirname $(which python3))/../"
!apt-get update && apt-get install -y ffmpeg libsm6 libxext6 git ninja-build libglib2.0-0 libsm6 libxrender-dev libxext6


env: FORCE_CUDA='1'
env: TORCH_CUDA_ARCH_LIST="6.0 6.1 7.0+PTX"
env: TORCH_NVCC_FLAGS="-Xfatbin -compress-all"
env: CMAKE_PREFIX_PATH="$(dirname $(which python3))/../"
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Pac

**Step 2**: install the proper version of PyTorch and Torchvision (:warning: may take 5 mins):          

In [5]:
print(f'-- Installing {CUDA} dependency...')
!pip uninstall torchtext torchaudio -y
!pip install --quiet -U torch=={t_version}+{CUDA} torchvision=={tv_version}+{CUDA} --find-links https://download.pytorch.org/whl/torch_stable.html

-- Installing cu111 dependency...
Found existing installation: torchtext 0.11.0
Uninstalling torchtext-0.11.0:
  Successfully uninstalled torchtext-0.11.0
Found existing installation: torchaudio 0.10.0+cu111
Uninstalling torchaudio-0.10.0+cu111:
  Successfully uninstalled torchaudio-0.10.0+cu111
     |█████████████▌                  | 834.1 MB 2.1 MB/s eta 0:09:10tcmalloc: large alloc 1147494400 bytes == 0x56504a338000 @  0x7fbca9abf615 0x56501005e4cc 0x56501013e47a 0x5650100612ed 0x565010152e1d 0x5650100d4e99 0x5650100cf9ee 0x565010062bda 0x5650100d4d00 0x5650100cf9ee 0x565010062bda 0x5650100d1737 0x565010153c66 0x5650100d0daf 0x565010153c66 0x5650100d0daf 0x565010153c66 0x5650100d0daf 0x565010063039 0x5650100a6409 0x565010061c52 0x5650100d4c25 0x5650100cf9ee 0x565010062bda 0x5650100d1737 0x5650100cf9ee 0x565010062bda 0x5650100d0915 0x565010062afa 0x5650100d0c0d 0x5650100cf9ee
     |█████████████████               | 1055.7 MB 1.3 MB/s eta 0:11:35tcmalloc: large alloc 1434370048 bytes 

In [ ]:
Check that verything has been installed properly:

In [6]:
# Vérification de l'installation de Pytorch
import torch, torchvision
print('-- Checking Pytorch installation...')
print(torchvision.__version__, torch.__version__, torch.cuda.is_available())

-- Checking Pytorch installation...
0.9.0+cu111 1.8.0+cu111 True


**Step 3**: install mmcv-full, note that can take a long time (up to 15 mins)

In [7]:

# Installation de mmcv-full afin d'utiliser les opérateurs CUDA
print('\n-- Installing OpenMMLab Computer Vision (mmcv-full)...')
!pip install mmcv-full -f https://download.openmmlab.com/mmcv/dist/{CUDA}/torch{t_version}/index.html


-- Installing OpenMMLab Computer Vision (mmcv-full)...
Looking in links: https://download.openmmlab.com/mmcv/dist/cu111/torch1.8.0/index.html
     |████████████████████████████████| 53.3 MB 2.6 MB/s 
     |████████████████████████████████| 185 kB 4.1 MB/s 


**Step 4**: install mmdetection

In [8]:
# Installation de MMDetection
print('\n-- Installing MMDetection...(v2.18.1)')
!rm -rf mmdetection
!git clone -b v2.18.1 https://github.com/open-mmlab/mmdetection.git
%cd mmdetection

!pip install --no-cache-dir --quiet -e .

# Installation de Pillow
print('\n-- Installing Pillow...')
!pip install --quiet Pillow==7.0.0


-- Installing MMDetection...(v2.18.1)
Cloning into 'mmdetection'...
remote: Enumerating objects: 22199, done.
remote: Total 22199 (delta 0), reused 0 (delta 0), pack-reused 22199
Receiving objects: 100% (22199/22199), 25.39 MiB | 13.65 MiB/s, done.
Resolving deltas: 100% (15548/15548), done.
Note: checking out 'c76ab0eb3c637b86c343d8454e07e00cfecc1b78'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

/content/mmdetection

-- Installing Pillow...
     |████████████████████████████████| 2.1 MB 4.1 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source 

Check that your configuration is working:

In [9]:
# Vérification de l'installation de MMDetection
import mmdet
print('\n-- Checking MMDetection installation...')
print(mmdet.__version__)

# Vérification de l'installation de mmcv
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print('\n-- Checking mmcv installation...')
print(get_compiling_cuda_version())
print(get_compiler_version())


-- Checking MMDetection installation...
2.18.1

-- Checking mmcv installation...
11.1
GCC 7.3


# Install Remote-SSH

see https://github.com/WassimBenzarti/colab-ssh#getting-started for more details.

You'll need to create a few account for Grok (https://ngrok.com/) and put your Token below.

In [10]:
!pip install colab_ssh --upgrade
from colab_ssh import launch_ssh
# in a separate cell in case you need to restart the service
ngrokToken = '<YOUR GROK TOKEN>'
launch_ssh(ngrokToken, password="password")

we highly recommend that update your code by following this documentation https://github.com/WassimBenzarti/colab-ssh#getting-started
Successfully running 8.tcp.ngrok.io:15005
[Optional] You can also connect with VSCode SSH Remote extension using this configuration:

  Host google_colab_ssh
    HostName 8.tcp.ngrok.io
    User root
    Port 15005
    


Add the config shown above in vscode (View/Command Palette/Remote-SSH: Open config). For instance:

```
Host google_colab_ssh
    HostName 8.tcp.ngrok.io
    User root
    Port 15005
    RemoteCommand cd /content
    LocalForward 127.0.0.1:8888 127.0.0.1:8888
```



Clone the notebooks:

In [ ]:
%cd /content/
!git clone -b code-reorg https://github.com/sfoucher/Masterclass-Biodiversity.git

Launch a jupyter server:

In [13]:
%cd /content/
!pip install jupyterlab
!nohup jupyter lab --ip='0.0.0.0' --port=8888 --no-browser --allow-root --notebook-dir='/content/'&

nohup: appending output to 'nohup.out'
